In [0]:
import os
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

In [0]:
#display files, to take a look:
for fname in os.listdir('/dbfs/output_dir/trade'):
    print(fname)

date=2020-08-05
date=2020-08-06

In [0]:
for fname in os.listdir('/dbfs/output_dir/trade/date=2020-08-05'):
    print(fname)

_SUCCESS
_committed_1033323681082687431
_started_1033323681082687431
part-00000-tid-1033323681082687431-0318a199-80dd-4852-a9c7-2ad21bdc45da-83-1-c000.snappy.parquet
part-00001-tid-1033323681082687431-0318a199-80dd-4852-a9c7-2ad21bdc45da-84-1-c000.snappy.parquet

In [0]:
for fname in os.listdir('/dbfs/output_dir/trade/date=2020-08-06'):
    print(fname)

_SUCCESS
_committed_263691496260930337
_started_263691496260930337
part-00000-tid-263691496260930337-9c4dfdc0-5ba8-4da8-a801-45f7dbe7b095-98-1-c000.snappy.parquet
part-00001-tid-263691496260930337-9c4dfdc0-5ba8-4da8-a801-45f7dbe7b095-99-1-c000.snappy.parquet

In [0]:
for fname in os.listdir('/dbfs/output_dir/quote/date=2020-08-06'):
    print(fname)

_SUCCESS
_committed_6750559710475587711
_started_6750559710475587711
part-00000-tid-6750559710475587711-f2508b5a-fec7-4d73-8e28-003f9e9ba647-91-1-c000.snappy.parquet

In [0]:
for fname in os.listdir('/dbfs/output_dir/trade/date=2020-08-05'):
    print(fname)

_SUCCESS
_committed_1033323681082687431
_started_1033323681082687431
part-00000-tid-1033323681082687431-0318a199-80dd-4852-a9c7-2ad21bdc45da-83-1-c000.snappy.parquet
part-00001-tid-1033323681082687431-0318a199-80dd-4852-a9c7-2ad21bdc45da-84-1-c000.snappy.parquet

In [0]:
#4.1 Read Parquet Files
#from load step: corrected.write.mode("append").parquet("c:/sb/equity-market-data-analysis/output_dir/trade/date={}".format(date))
tradedf = spark.read.parquet("/output_dir/trade")
tradedf.createOrReplaceTempView("trades")
tradedf.show(5)

quotedf = spark.read.parquet("/output_dir/quote")
quotedf.createOrReplaceTempView("quotes")
quotedf.show(5)

+----------+------+--------+--------------------+------------+-------------------+---------+-------------------+----------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb| arrival_tm| trade_pr| latest| date|
+----------+------+--------+--------------------+------------+-------------------+---------+-------------------+----------+
2020-08-06| SYMB| NYSE|2020-08-06 16:57:...| 60|2020-08-06 09:30:00| 33.31802|2020-08-06 09:30:00|2020-08-06|
2020-08-06| SYMB| NYSE|2020-08-06 18:06:...| 70|2020-08-06 09:30:00|33.565834|2020-08-06 09:30:00|2020-08-06|
2020-08-06| SYMB| NYSE|2020-08-06 19:21:...| 80|2020-08-06 09:30:00| 33.10892|2020-08-06 09:30:00|2020-08-06|
2020-08-06| SYMB| NYSE|2020-08-06 20:36:...| 90|2020-08-06 09:30:00|32.643288|2020-08-06 09:30:00|2020-08-06|
2020-08-06| SYMB| NYSE|2020-08-06 21:46:...| 100|2020-08-06 09:30:00|35.924744|2020-08-06 09:30:00|2020-08-06|
+----------+------+--------+--------------------+------------+-------------------+---------+-------------------+----------+
only showing top 5 rows

+----------+------+--------+--------------------+------------+-------------------+---------+--------+---------+--------+-------------------+----------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb| arrival_tm| bid_pr|bid_size| ask_pr|ask_size| latest| date|
+----------+------+--------+--------------------+------------+-------------------+---------+--------+---------+--------+-------------------+----------+
2020-08-06| SYMB| NYSE|2020-08-06 14:24:...| 39|2020-08-06 09:30:00|31.945055| 100|33.670525| 100|2020-08-06 09:30:00|2020-08-06|
2020-08-06| SYMB| NYSE|2020-08-06 12:09:...| 21|2020-08-06 09:30:00| 34.82692| 100|36.231388| 100|2020-08-06 09:30:00|2020-08-06|
2020-08-06| SYMA| NYSE|2020-08-06 16:33:...| 57|2020-08-06 09:30:00|75.559685| 100| 75.80727| 100|2020-08-06 09:30:00|2020-08-06|
2020-08-06| SYMA| NYSE|2020-08-06 16:22:...| 55|2020-08-06 09:30:00| 79.20979| 100| 80.53262| 100|2020-08-06 09:30:00|2020-08-06|
2020-08-06| SYMA| NYSE|2020-08-06 21:04:...| 92|2020-08-06 09:30:00| 75.75031| 100| 77.66744| 100|2020-08-06 09:30:00|2020-08-06|
+----------+------+--------+--------------------+------------+-------------------+---------+--------+---------+--------+-------------------+----------+
only showing top 5 rows

In [0]:
#4.2 Create trade staging table
#4.2.1 use spark to read the trade table with date partition "2020-08-05'
query = """
    select trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr from trades
    where trade_dt = '2020-08-06'
    """
df = spark.sql(query)
#4.2.2 Create a spark temporary view
df.createOrReplaceTempView("tmp_trade_moving_avg1")

In [0]:
df.show(5)

+----------+------+--------+--------------------+------------+---------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb| trade_pr|
+----------+------+--------+--------------------+------------+---------+
2020-08-06| SYMB| NYSE|2020-08-06 16:57:...| 60| 33.31802|
2020-08-06| SYMB| NYSE|2020-08-06 18:06:...| 70|33.565834|
2020-08-06| SYMB| NYSE|2020-08-06 19:21:...| 80| 33.10892|
2020-08-06| SYMB| NYSE|2020-08-06 20:36:...| 90|32.643288|
2020-08-06| SYMB| NYSE|2020-08-06 21:46:...| 100|35.924744|
+----------+------+--------+--------------------+------------+---------+
only showing top 5 rows

In [0]:
#4.2.3 Calculate The 30-min Moving Average using tmp_trade_moving_avg
query = """
    SELECT trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr,
    avg(trade_pr) OVER(PARTITION BY symbol ORDER BY CAST(event_tm AS timestamp) 
        RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as mov_avg_pr
    FROM tmp_trade_moving_avg1
    """
mov_avg_df = spark.sql(query)

In [0]:
mov_avg_df.show(5)

+----------+------+--------+--------------------+------------+---------+-----------------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb| trade_pr| mov_avg_pr|
+----------+------+--------+--------------------+------------+---------+-----------------+
2020-08-06| SYMA| NYSE|2020-08-06 10:49:...| 10|74.493774| 74.4937744140625|
2020-08-06| SYMA| NYSE|2020-08-06 12:00:...| 20|76.161194|76.16119384765625|
2020-08-06| SYMA| NYSE|2020-08-06 13:11:...| 30|76.899635| 76.8996353149414|
2020-08-06| SYMA| NYSE|2020-08-06 14:27:...| 40| 77.11552|77.11551666259766|
2020-08-06| SYMA| NYSE|2020-08-06 15:39:...| 50| 76.37442|76.37442016601562|
+----------+------+--------+--------------------+------------+---------+-----------------+
only showing top 5 rows

In [0]:
#4.2.4 Save the temporary view into Hive table for staging

#mov_avg_df.write.saveAsTable("tmp_trade_moving_avg", mode='overwrite')
mov_avg_df.createOrReplaceTempView("tmp_trade_moving_avg")

In [0]:
#4.3 Create Staging Table for the prior day's last trade
#4.3.1 Get the previous date value
date = datetime.strptime('2020-08-06', '%Y-%m-%d')
print(date)
prev_date_str = str(date.date() - timedelta(days=1))
print(prev_date_str)

#4.3.2 Use Spark to read the trade table with date partition prev_date_str 
query = """
    select trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr from trades
    where trade_dt = '{}'
    """
df = spark.sql(query.format(prev_date_str))
df.show(5)

2020-08-06 00:00:00
2020-08-05
+----------+------+--------+--------------------+------------+---------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb| trade_pr|
+----------+------+--------+--------------------+------------+---------+
2020-08-05| SYMB| NYSE|2020-08-05 16:48:...| 60|35.762054|
2020-08-05| SYMB| NYSE|2020-08-05 18:02:...| 70|32.681087|
2020-08-05| SYMB| NYSE|2020-08-05 19:08:...| 80| 34.27408|
2020-08-05| SYMB| NYSE|2020-08-05 20:17:...| 90| 34.2493|
2020-08-05| SYMB| NYSE|2020-08-05 21:27:...| 100|33.956287|
+----------+------+--------+--------------------+------------+---------+
only showing top 5 rows

In [0]:
#4.3.3 Create spark temporary view
df.createOrReplaceTempView("tmp_last_trade1")

In [0]:
#4.3.4 Calculate last trade price using tmp_last_trade
query = """
    SELECT symbol, exchange, trade_pr AS close_pr
    FROM tmp_last_trade1 t1
    JOIN
    (SELECT MAX(event_tm) AS last_record FROM tmp_last_trade1
     GROUP BY symbol, exchange) t2
    ON t1.event_tm == last_record
    """

In [0]:
last_pr_df = spark.sql(query)

In [0]:
last_pr_df.show(5)

+------+--------+---------+
symbol|exchange| close_pr|
+------+--------+---------+
 SYMB| NYSE|33.956287|
 SYMC| NYSE|160.61949|
 SYMA| NYSE| 77.78611|
+------+--------+---------+

In [0]:
last_pr_df.createOrReplaceTempView("tmp_last_trade")

In [0]:
#4.4.1.2 Create spark temp view to union both tables
quote_union = spark.sql("""
    SELECT trade_dt, symbol, exchange, event_tm, event_seq_nb, NULL as arrival_tm, NULL as bid_size, 
        NULL as ask_pr, NULL as ask_size, NULL as latest, trade_pr, mov_avg_pr 
    FROM tmp_trade_moving_avg
    UNION 
    SELECT trade_dt, symbol, exchange, event_tm, event_seq_nb, arrival_tm, bid_size, ask_pr, ask_size,
        latest, NULL as trade_pr, NULL as mov_avg_pr 
    FROM quotes
    """)
quote_union.createOrReplaceTempView("quote_union")
quote_union.show(10)

+----------+------+--------+--------------------+------------+----------+--------+------+--------+------+---------+-----------------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb|arrival_tm|bid_size|ask_pr|ask_size|latest| trade_pr| mov_avg_pr|
+----------+------+--------+--------------------+------------+----------+--------+------+--------+------+---------+-----------------+
2020-08-06| SYMA| NYSE|2020-08-06 14:27:...| 40| null| null| null| null| null| 77.11552|77.11551666259766|
2020-08-06| SYMA| NYSE|2020-08-06 16:58:...| 60| null| null| null| null| null| 78.32364|78.32363891601562|
2020-08-06| SYMA| NYSE|2020-08-06 18:14:...| 70| null| null| null| null| null| 78.23471|78.23471069335938|
2020-08-06| SYMA| NYSE|2020-08-06 15:39:...| 50| null| null| null| null| null| 76.37442|76.37442016601562|
2020-08-06| SYMA| NYSE|2020-08-06 22:00:...| 100| null| null| null| null| null| 76.30847| 76.3084716796875|
2020-08-06| SYMA| NYSE|2020-08-06 13:11:...| 30| null| null| null| null| null|76.899635| 76.8996353149414|
2020-08-06| SYMA| NYSE|2020-08-06 10:49:...| 10| null| null| null| null| null|74.493774| 74.4937744140625|
2020-08-06| SYMA| NYSE|2020-08-06 19:28:...| 80| null| null| null| null| null|76.528984|76.52898406982422|
2020-08-06| SYMB| NYSE|2020-08-06 10:45:...| 10| null| null| null| null| null| 33.85934|33.85934066772461|
2020-08-06| SYMA| NYSE|2020-08-06 20:49:...| 90| null| null| null| null| null| 75.71484| 75.71484375|
+----------+------+--------+--------------------+------------+----------+--------+------+--------+------+---------+-----------------+
only showing top 10 rows

In [0]:
#4.4 Populate the latest trade_pr and mov_avg_pr 
quote_union_update = spark.sql("""
SELECT quote_union.trade_dt, quote_union.symbol, quote_union.exchange, quote_union.event_tm, 
        quote_union.event_seq_nb, quote_union.arrival_tm, quote_union.bid_size, quote_union.ask_pr, 
        quote_union.ask_size, quote_union.latest, latest_trade.latest_trade_pr, latest_trade.mov_avg_pr
FROM quote_union
JOIN (SELECT quote.symbol, quote.exchange, quote.trade_pr as latest_trade_pr, quote.mov_avg_pr
    FROM quote_union quote
    JOIN (SELECT MAX(event_tm) as latest_event_tm FROM quote_union GROUP BY symbol, exchange) last_trade_tm
        ON quote.event_tm == last_trade_tm.latest_event_tm) latest_trade
    ON quote_union.symbol == latest_trade.symbol AND quote_union.exchange == latest_trade.exchange
    """)
quote_union_update.createOrReplaceTempView("quote_union_update")
quote_union_update.show(10)

+----------+------+--------+--------------------+------------+-------------------+--------+---------+--------+-------------------+---------------+----------------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb| arrival_tm|bid_size| ask_pr|ask_size| latest|latest_trade_pr| mov_avg_pr|
+----------+------+--------+--------------------+------------+-------------------+--------+---------+--------+-------------------+---------------+----------------+
2020-08-05| SYMA| NYSE|2020-08-05 13:49:...| 35|2020-08-05 09:30:00| 100| 75.41533| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 10:11:...| 6|2020-08-05 09:30:00| 100|80.329056| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 13:57:...| 36|2020-08-05 09:30:00| 100| 78.38394| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 21:15:...| 98|2020-08-05 09:30:00| 100| 78.33672| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 17:13:...| 65|2020-08-05 09:30:00| 100| 76.75044| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 15:07:...| 47|2020-08-05 09:30:00| 100|76.427155| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 17:25:...| 67|2020-08-05 09:30:00| 100| 76.67752| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 16:16:...| 57|2020-08-05 09:30:00| 100| 77.49039| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 16:50:...| 62|2020-08-05 09:30:00| 100| 80.96721| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 13:03:...| 29|2020-08-05 09:30:00| 100| 79.5748| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
+----------+------+--------+--------------------+------------+-------------------+--------+---------+--------+-------------------+---------------+----------------+
only showing top 10 rows

In [0]:
#4.4.1.4 Filter for quote records
quote_update = spark.sql("""
SELECT *
FROM quote_union_update
WHERE ask_size IS NOT NULL
""")
quote_update.createOrReplaceTempView("quote_update")
quote_update.show(5)

+----------+------+--------+--------------------+------------+-------------------+--------+---------+--------+-------------------+---------------+----------------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb| arrival_tm|bid_size| ask_pr|ask_size| latest|latest_trade_pr| mov_avg_pr|
+----------+------+--------+--------------------+------------+-------------------+--------+---------+--------+-------------------+---------------+----------------+
2020-08-05| SYMA| NYSE|2020-08-05 13:49:...| 35|2020-08-05 09:30:00| 100| 75.41533| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 10:11:...| 6|2020-08-05 09:30:00| 100|80.329056| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 13:57:...| 36|2020-08-05 09:30:00| 100| 78.38394| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 21:15:...| 98|2020-08-05 09:30:00| 100| 78.33672| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
2020-08-05| SYMA| NYSE|2020-08-05 17:13:...| 65|2020-08-05 09:30:00| 100| 76.75044| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|
+----------+------+--------+--------------------+------------+-------------------+--------+---------+--------+-------------------+---------------+----------------+
only showing top 5 rows

In [0]:
#4.4.2 Join with table temp_last_trade to get the prior day close price
quote_final = spark.sql("""
SELECT * 
FROM quote_update 
LEFT JOIN tmp_last_trade
    USING(symbol, exchange)
""")
quote_final.show(5)

+------+--------+----------+--------------------+------------+-------------------+--------+---------+--------+-------------------+---------------+----------------+--------+
symbol|exchange| trade_dt| event_tm|event_seq_nb| arrival_tm|bid_size| ask_pr|ask_size| latest|latest_trade_pr| mov_avg_pr|close_pr|
+------+--------+----------+--------------------+------------+-------------------+--------+---------+--------+-------------------+---------------+----------------+--------+
 SYMA| NYSE|2020-08-05|2020-08-05 13:49:...| 35|2020-08-05 09:30:00| 100| 75.41533| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|77.78611|
 SYMA| NYSE|2020-08-05|2020-08-05 10:11:...| 6|2020-08-05 09:30:00| 100|80.329056| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|77.78611|
 SYMA| NYSE|2020-08-05|2020-08-05 13:57:...| 36|2020-08-05 09:30:00| 100| 78.38394| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|77.78611|
 SYMA| NYSE|2020-08-05|2020-08-05 21:15:...| 98|2020-08-05 09:30:00| 100| 78.33672| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|77.78611|
 SYMA| NYSE|2020-08-05|2020-08-05 17:13:...| 65|2020-08-05 09:30:00| 100| 76.75044| 100|2020-08-05 09:30:00| 76.30847|76.3084716796875|77.78611|
+------+--------+----------+--------------------+------------+-------------------+--------+---------+--------+-------------------+---------------+----------------+--------+
only showing top 5 rows

In [0]:
#4.4.3 Write the final dataframe into Azure Blob storage at corresponding partition
#temp local storage for jupyter notebook
#quote_final.write.parquet("c:/sb/equity-market-data-analysis/output_dir/quote-trade-analytical/date=2020-08-06")

storageAccountName = 'equitymarketblob'
storageAccountAccessKey = '0nboKYno6oyL7Op44qYarGAfeq1EmdnrBUHWvYUyfDfIdGF990OzOw8KsL7RdEofxLC7W0WaHB66+AStrlJdZA=='
ContainerName = 'bcontainer1'

spark.conf.set(f'fs.azure.account.key.{storageAccountName}.blob.core.windows.net', storageAccountAccessKey)
path='wasbs://{}@{}.blob.core.windows.net/output_dir/quote-trade-analytical/date=2020-08-06/'.format(ContainerName,storageAccountName)
df.write.parquet(path)